# Load Libraries

In [1]:
import os
import glob

import pandas as pd
import numpy as np
from pathlib import Path
import copy

import random
from tqdm.notebook import tqdm
import pydicom # Handle MRI images
from pydicom.pixel_data_handlers.util import apply_voi_lut

import cv2  # OpenCV - https://docs.opencv.org/master/d6/d00/tutorial_py_root.html

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import matplotlib
import torch

2021-10-22 00:46:16.688095: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing import image


IMAGE_SIZE = 128
base_resnet = Xception(    
    weights= '../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5',
    pooling='avg',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False)
#base_resnet.load_weights('../input/NASNet-large-no-top/NASNet-large-no-top.h5')
#base_resnet.summary()


base_resnet.trainable = False

2021-10-22 00:46:25.473610: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-22 00:46:25.476970: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-22 00:46:25.516625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 00:46:25.517290: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-22 00:46:25.517358: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-22 00:46:25.543868: I tensorflow/stream_executor/platform/def

# Configuration, Constants, Setup

In [4]:
data_dir = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/')
mri_types = ["FLAIR", "T1w", "T2w", "T1wCE"]
excluded_images = [109, 123, 709] # Bad images

# Load Datasets

In [5]:
train_df = pd.read_csv(data_dir / "train_labels.csv",
#                        index='id',
#                       nrows=100000
                      )
test_df = pd.read_csv(data_dir / "sample_submission.csv")
sample_submission = pd.read_csv(data_dir / "sample_submission.csv")

#padnda dataframe with one coloumn ID the other the MGMT presence (1 yes 0 no)
train_df = train_df[~train_df.BraTS21ID.isin(excluded_images)]

print(f"train data: Rows={train_df.shape[0]}, Columns={train_df.shape[1]}")
# print(f"test data : Rows={test_df.shape[0]}, Columns={test_df.shape[1]}")

train data: Rows=582, Columns=2


# Utility Functions

### There's a version that converts into grayscale: 

- https://www.kaggle.com/smoschou55/advanced-eda-brain-tumor-data


In [6]:
def load_dicom(path, voi_lut=True, fix_monochrome=True, remove_black_boundary=True, augmentation=False):
    global IMAGE_SIZE
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    #if remove_black_boundary: # we get slightly more details
    #    (x, y) = np.where(data > 0)
    #    if len(x) > 0 and len(y) > 0:
    #        x_mn = np.min(x)
    #        x_mx = np.max(x)
    #        y_mn = np.min(y)
    #        y_mx = np.max(y)
    #        if (x_mx - x_mn) > 10 and (y_mx - y_mn) > 10:
    #            data = data[:,np.min(y):np.max(y)]
    
    #check if the image is all black:
    if np.sum(data) < 0.01:
        #print("LOADING BLACK IMAGE")
        remove_black_boundary = False
        
   
    if remove_black_boundary:
        data_no_boundary_1 = []
        data_no_boundary_2 = []
        #remove black rows
        for i in range(len(data)):
            if np.sum(data[i]) > 0.01:
                data_no_boundary_1.append(data[i])
        data_no_boundary_1 = np.array(data_no_boundary_1)
        for j in range(np.shape(data_no_boundary_1)[1]):
            if np.sum(data_no_boundary_1[:,j]) != 0:
                data_no_boundary_2.append(data_no_boundary_1[:,j])
        data_no_boundary_2 = np.array(data_no_boundary_2).T
        data = data_no_boundary_2
        
    data = np.array(data)
    data = cv2.resize(data, (IMAGE_SIZE, IMAGE_SIZE))
    data = cv2.cvtColor(data,cv2.COLOR_GRAY2RGB)

    return data, remove_black_boundary

In [7]:
sometimes = lambda aug: iaa.Sometimes(0.2, aug)

seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.05),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.LinearContrast((0.5, 2.0))
                    )
                ]),
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)
#slices_aug = seq(images=slices.T)

/opt/conda/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `SimplexNoiseAlpha()` is deprecated. Use `BlendAlphaSimplexNoise` instead. SimplexNoiseAlpha is deprecated. Use BlendAlphaSimplexNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)
/opt/conda/lib/python3.7/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)


In [8]:
radius = 60
def get_all_image_paths(brats21id, image_type, folder='train'): 
    '''
    Returns an arry of all the images of a particular type for a particular patient ID
    '''
    assert(image_type in mri_types)
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    middle = num_images//2
    
    return (paths)

def get_all_images(brats21id, image_type, folder='train'):
    images = []
    for path in get_all_image_paths(brats21id, image_type, folder):
        image, remove_black = load_dicom(path)
        if remove_black == True:
            images.append(image)
            
    if len(images) > radius:
        indices = random.sample(range(len(images)), radius)
        images = [images[i] for i in sorted(indices)]
    else:
        m = int(radius/len(images))
        rest = radius - m*len(images)
        images_ = copy.copy(images)
      
        for k in range(1, m):
            j = 0
            for image in images_:
                images.insert(j*(k+1), seq(images = image)) 
                j +=1
        
        for _ in range(rest):
            rndm = random.randint(0, len(images_)-1)
            images.insert(rndm, seq(images = images[rndm]))
    return images

# Load Images We Will Need + Feature Extraction

In [9]:
def get_all_data_for_train(image_size=IMAGE_SIZE):
    global train_df
    global mri_types
    global base_resnet
    y = []
    train_ids = []
    listMatrix = []
    for i in tqdm(train_df.index):
        #print("INIZIO CICLO: ", i)
        img_s = []
        for mri in mri_types:
            #print("TIPO DI IMMAGINE: ", mri)
            x = train_df.loc[i]
            #here we have all the images that have just one ID. We want more, we can performe data augmentation. 
            images = get_all_images(int(x['BraTS21ID']), mri, 'train')
            #print("IMAGE")
            #print(mri)
            #print(np.shape(images[0]))
            #plt.imshow(images[0])
            #plt.show()
            #print("\n")
            img_s += images
            #print("NUMERO IMMAGINI xTIPO xPAZIENTE DOPO AUMENTO: ", len(images))
            #images_augmented = perform_data_augmentation(images)
        label = x['MGMT_value']
        IMGS = np.array(img_s)
        IMGS = tf.expand_dims(IMGS, axis = -1)
        #IMGS = tf.keras.applications.xception.preprocess_input(IMGS)
        PatientMatrix = base_resnet.predict(IMGS)
        print(np.shape(PatientMatrix))
        listMatrix.append(PatientMatrix)

        y.append(label)
        train_ids.append(int(x['BraTS21ID']))
    assert(len(listMatrix) == len(y))
    return listMatrix, y, train_ids 

In [10]:
listMatrix, y, train_ids = get_all_data_for_train(image_size = IMAGE_SIZE)

  0%|          | 0/582 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/opt/conda/lib/python3.7/site-packages/imgaug/augmenters/base.py:38: SuspiciousMultiImageShapeWarning: You provided a numpy array of shape (128, 128, 3) as a multi-image augmentation input, which was interpreted as (N, H, W). The last dimension however has value 1 or 3, which indicates that you provided a single image with shape (H, W, C) instead. If that is the case, you should use e.g. augmenter(image=<your input>) or augment_image(<your input>) -- note the singular 'image' instead of 'imageS'. Otherwise your single input image will be interpreted as multiple images of shape (H, W) during augmentation.
  category=SuspiciousMultiImageShapeWarning)
/opt/conda/lib/python3.7/site-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely 

(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240, 2048)
(240

In [11]:
"""inputs = kr.Input(shape=input_shape, name='input')

x = layers.Conv2D(filters, (3, 3), activation='relu', strides=2, padding='same')(inputs)
x = layers.Conv2D(filters*2, (3, 3), activation='relu', strides=2, padding='same')(x)

# shape info needed to build decoder model
shape = x.get_shape().as_list()

# generate latent vector Q(z|X)
x = layers.Flatten()(x)
x = layers.Dense(16, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

z = layers.Lambda(sampling, name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = kr.Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()"""

"inputs = kr.Input(shape=input_shape, name='input')\n\nx = layers.Conv2D(filters, (3, 3), activation='relu', strides=2, padding='same')(inputs)\nx = layers.Conv2D(filters*2, (3, 3), activation='relu', strides=2, padding='same')(x)\n\n# shape info needed to build decoder model\nshape = x.get_shape().as_list()\n\n# generate latent vector Q(z|X)\nx = layers.Flatten()(x)\nx = layers.Dense(16, activation='relu')(x)\nz_mean = layers.Dense(latent_dim, name='z_mean')(x)\nz_log_var = layers.Dense(latent_dim, name='z_log_var')(x)\n\nz = layers.Lambda(sampling, name='z')([z_mean, z_log_var])\n\n# instantiate encoder model\nencoder = kr.Model(inputs, [z_mean, z_log_var, z], name='encoder')\nencoder.summary()"

In [12]:
# PRocess feature vectors
X_2 = np.asarray(listMatrix)
y_ = np.asarray(y)
_train_ids = np.array(train_ids)
X_train, X_valid, y_train, y_valid, ids_train, ids_valid = train_test_split(X_2, y_, _train_ids, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
X_train = tf.expand_dims(X_train, axis = -1)
X_valid = tf.expand_dims(X_valid, axis = -1)
X_ = tf.expand_dims(X_2, axis = -1)

2021-10-22 04:17:28.146967: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 914227200 exceeds 10% of free system memory.
2021-10-22 04:17:29.521834: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1144258560 exceeds 10% of free system memory.


In [13]:
def get_all_data_for_test(image_size=IMAGE_SIZE):
    global test_df
    global mri_types
    global base_resnet
    test_ids = []
    test_listMatrix = []
    for i in tqdm(test_df.index):
        img_s = []
        for mri in mri_types:
            x = test_df.loc[i]
            images = get_all_images(int(x['BraTS21ID']), mri, 'test')
            img_s += images
        label = x['MGMT_value']
        IMGS = np.array(img_s)
        IMGS = tf.expand_dims(IMGS, axis = -1)
                
        test_PatientMatrix = base_resnet.predict(IMGS)
        test_listMatrix.append(test_PatientMatrix)
        test_ids.append(int(x['BraTS21ID']))

    return test_listMatrix, test_ids 

In [14]:
test_listMatrix, test_ids = get_all_data_for_test(image_size = IMAGE_SIZE)

  0%|          | 0/87 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [15]:
X_2_test = np.asarray(test_listMatrix)
print(np.shape(X_2_test))
# PRocess feature vectors
_test_ids = np.array(test_ids)
X_test= tf.expand_dims(X_2_test, axis = -1)

(87, 240, 2048)


# Models

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

X_for_voting = X_.numpy().reshape(X_.numpy().shape[0], -1)
X_test_voting = X_test.numpy().reshape(X_test.numpy().shape[0], -1)

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=50, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42,probability=True)
knn_clf = KNeighborsClassifier(n_neighbors=5)
ab_clf = AdaBoostClassifier(n_estimators=50, random_state=42)
voting = VotingClassifier(
             estimators=[#('lr', log_clf),
                         ('rf', rnd_clf),
                         #('svc', svm_clf),
                         #('knn',knn_clf),
                         ('ab', ab_clf)
                        ],
             voting='soft',
             flatten_transform=True)

voting.fit(X_for_voting,y_)

2021-10-22 04:49:24.504012: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1144258560 exceeds 10% of free system memory.


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=42)),
                             ('ab', AdaBoostClassifier(random_state=42))],
                 voting='soft')

# Predictions on the Test Set

In [17]:
pred = voting.predict(X_test_voting)
#pred = np.argmax(y_pred, axis=1) #

result = pd.DataFrame(test_ids)
result[1] = pred
pred

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [18]:
result.columns=['BraTS21ID','MGMT_value']

result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample_submission['BraTS21ID']

# Rounding... 0.907866 -> 0.9
result2['MGMT_value'] = result2['MGMT_value'].apply(lambda x:round(x*10)/10)
# result2['MGMT_value'] = result2['MGMT_value'] # No rounding
result2.to_csv('submission.csv',index=False)
result2

,BraTS21ID,MGMT_value
0,1,1.0
1,13,1.0
2,15,1.0
3,27,1.0
4,37,1.0
...,...,...
82,826,0.0
83,829,0.0
84,833,0.0
85,997,0.0
